In [ ]:
# | default_exp docstring_generator

# Docstring Generator

In [ ]:
# | export

import ast
import tokenize
import os
from typing import *
from pathlib import Path
from io import BytesIO
from configparser import ConfigParser

import nbformat
import openai

from docstring_gen._helpers.utils import completions_with_backoff

In [ ]:
import shutil
from tempfile import TemporaryDirectory
from contextlib import contextmanager

import pytest

In [ ]:
# | export

def _get_code_from_source(source: str, start_line_no: int, end_line_no: int) -> str:
    """    Extracts the code from the source file.
     
        Args:
            source: The source code of the file.
            start_line_no: The line number from which the code is to be extracted.
            end_line_no: The line number till which the code is to be extracted.
     
        Returns:
            The code extracted from the source file.


    Note: The above docstring is autogenerated by docstring-gen library!
    """
    source_lines = source.split("\n")
    extracted_lines = source_lines[start_line_no-1:end_line_no]
    return "\n".join(extracted_lines)

In [ ]:
source = """
class test:
    CONST_VAL = 1
    def __init__(self, a):
        self.a = a
        
    async def drive(self):
        print(f'The {self.model} is now driving.')
"""
start_line_no = 7
end_line_no = 8

expected = """    async def drive(self):
        print(f'The {self.model} is now driving.')"""

actual = _get_code_from_source(source, start_line_no, end_line_no)
print(actual)

assert actual == expected


    async def drive(self):
        print(f'The {self.model} is now driving.')


In [ ]:
# | export


def _calculate_end_lineno(source: str, start_line_no: int) -> int:
    """Calculates the end line number of a block of code.
    
    Args:
        source: The source code of the file.
        start_line_no: The line number of the start of the block.
    
    Returns:
        The line number of the end of the block.


    Note: The above docstring is autogenerated by docstring-gen library!
    """
    lines = source.split("\n")[start_line_no - 1 :]
    first_indent = len(lines[0]) - len(lines[0].lstrip())
    end_line_in_source = 0

    for i, line in enumerate(lines[1:]):
        if len(line) - len(line.lstrip()) == first_indent and line.strip() != "":
            end_line_in_source = i
            break

    ret_val = (
        len(source.split("\n"))
        if end_line_in_source == 0
        else end_line_in_source + start_line_no
    )
    return ret_val - 1

In [ ]:
source = """
class A:
    CONST_VAL = 1
    def __init__(self, a):
        self.a = a
        
    async def drive(self):
        print(f'The {self.model} is now driving.')

class B:
    CONST_VAL = 1
    def __init__(self, a):
        self.a = a
        
    async def drive(self):
        print(f'The {self.model} is now driving.')
        
    async def stop(self):
        print(f'The {self.model} is now stopped.')
        
class C:
    CONST_VAL = 1
    def __init__(self, a):
        self.a = a

    async def drive(self):
        print(f'The {self.model} is now driving.')
        
async def drive(self):
    print(f'The {self.model} is now driving.')
"""

tree = ast.parse(source)
expected = [8, 5, 9, 19, 13, 16, 20, 27, 24, 28, 30]
actual = []
for i, node in enumerate(tree.body):
    res = _calculate_end_lineno(source, node.lineno)
    actual.append(res)
#     print(node.end_lineno)

    if isinstance(node, ast.ClassDef):
        for f in node.body:
            if not isinstance(f, (ast.FunctionDef, ast.AsyncFunctionDef)):
                continue
            res = _calculate_end_lineno(source, f.lineno)
            actual.append(res)
#             print(f.end_lineno)
print(actual)
assert actual == expected

[8, 5, 9, 19, 13, 16, 20, 27, 24, 28, 30]


In [ ]:
# | export


def _line_has_decorator(source: str, lineno: int) -> bool:
    """Checks if a line in a source file has a decorator.
     
    Args:
        source: The source code of the file.
        lineno: The line number to check.
     
    Returns:
        True if the line has a decorator, False otherwise.


    Note: The above docstring is autogenerated by docstring-gen library!
    """
    line = "".join(source.split("\n")[lineno - 1])
    return line.startswith("@") or line.strip() == ""

def _get_start_line_for_class_or_func(source: str, lineno: int) -> int:
    """    Returns the line number of the first line of the class or function definition.
        This is useful for getting the line number of the class or function definition
        when the line number of a decorator is given.
     
        Args:
            source: The source code of the file.
            lineno: The line number of the decorator.
     
        Returns:
            The line number of the first line of the class or function definition.


    Note: The above docstring is autogenerated by docstring-gen library!
    """
    if not _line_has_decorator(source, lineno):
        return lineno

    original_lineno = lineno
    total_lines = source.split("\n")
    for i in total_lines:
        lineno += 1
        if lineno > len(total_lines):
            break
        if not _line_has_decorator(source, lineno):
            return lineno
    return original_lineno

In [ ]:
source = """def decorator1(func):
    \"""Sample docstring

    Args:
        s: sample args

    Returns:
        sample return
    \"""
    def inner():
        func()
    return inner

def decorator2(func):
    \"""Sample docstring

    Args:
        s: sample args

    Returns:
        sample return
    \"""
    def inner():
        func()
    return inner


@decorator1
@decorator2
def outer_func():
    def inner_func():
        print("Hello, World!")
    inner_func()
"""

line_no = 28
expected = 30
actual = _get_start_line_for_class_or_func(source, line_no)
print(actual)
assert actual == expected

30


In [ ]:
source = """
async def drive(self
):
    print(f'The {self.model} is now driving.')
"""
lineno = 2
expected = 2
actual = _get_start_line_for_class_or_func(source, lineno)
print(actual)

assert actual == expected

source = """
@foo(x=5)
@decorator2
async def drive(self):
    print(f'The {self.model} is now driving.')
"""
lineno = 2
expected = 4
actual = _get_start_line_for_class_or_func(source, lineno)
print(actual)

assert actual == expected

source = """
@foo(x=5)
async def drive(self):
    print(f'The {self.model} is now driving.')
"""
lineno = 2
expected = 3
actual = _get_start_line_for_class_or_func(source, lineno)
print(actual)

assert actual == expected

source = """
@foo(x=5)
@bar(x=5)
@zar(x=5)
class A:
    CONST_VAL = 1
    def __init__(self, a):
        self.a = a
        
    async def drive(self):
        print(f'The {self.model} is now driving.')
"""
lineno = 2
expected = 5
actual = _get_start_line_for_class_or_func(source, lineno)
print(actual)

assert actual == expected

source = """
@foo(x=5)
@bar(x=5)

"""
lineno = 2
expected = 2
actual = _get_start_line_for_class_or_func(source, lineno)
print(actual)

assert actual == expected, f"{actual=}"



2
4
3
5
2


In [ ]:
# | export

def _get_lineno_to_append_docstring(source: str, lineno: int) -> int:
    """    This function takes a source code string and a line number and returns the line number
        where the docstring should be appended.
        This is done by tokenizing the source code and checking if the tokenization is successful.
        If it is not successful, the line number is incremented and the tokenization is tried again.
        This is done until the tokenization is successful.
        The line number where the tokenization is successful is returned.
        If the tokenization is never successful, a TokenError is raised.


    Note: The above docstring is autogenerated by docstring-gen library!
    """
    line_offset = 0
    is_src_tokenized = False
    lines = source.split("\n")[lineno - 1:]

    for i in range(len(lines)):
        line = "".join(source.split("\n")[lineno - 1:][:i+1])
        if line != "":
            try:
                list(tokenize.tokenize(BytesIO(line.encode("utf-8")).readline))
                is_src_tokenized = True
                break
            except tokenize.TokenError as e:
                line_offset +=1
                continue
    if not is_src_tokenized:
        raise tokenize.TokenError(f"TokenError: {source}")
    
    ret_val = line_offset + lineno
    return ret_val

In [ ]:
source = """
def gen(
    path: str = typer.Option(
      ...,
        help="The path to the Jupyter notebook or Python file, or a directory containing these files",
    ),
    prompt: Optional[str] = typer.Option(
        None,
 help="Text that will be given as input to the GPT-3 model to generate the docstring. If no text is provided, the docstring will be generated according to the Google Python Style Guide.",
    ),
    include_auto_gen_txt: bool = typer.Option(
        True,
        help="If set to true, a note indicating that the docstring was autogenerated will be added to the end of the docstring.",
    ),
    model: str = typer.Option(
        "code-davinci-002",
        help="The name of the GPT-3 model to use for docstring generation.",
    ),
    temperature: int= typer.Option(
        0, help="The temperature parameter for the GPT-3 model."
    ),
    max_tokens: int = typer.Option(
        250, help="The maximum number of tokens to generate in the docstring."
    ),
    top_p: float = typer.Option(1.0, help="The top-p parameter for the GPT-3 model."),
    frequency_penalty: float = typer.Option(
        0.0, help="The frequency penalty parameter for the GPT-3 model."
    ),
    presence_penalty: float = typer.Option(
        0.0, help="The presence penalty parameter for the GPT-3 model."
    )
):
    pass
"""
lineno = 2
expected = 32

actual = _get_lineno_to_append_docstring(source, lineno)
print(actual)

assert actual == expected

32


In [ ]:
source = """
async def drive(self):
    print(f'The {self.model} is now driving.')
"""
lineno = 2
expected = 2

actual = _get_lineno_to_append_docstring(source, lineno)
print(actual)

assert actual == expected

source = """
async def drive(self,
a,
b,
c,
"""
lineno = 2
with pytest.raises(tokenize.TokenError) as e:
    _get_lineno_to_append_docstring(source, lineno)
print(e.value)

2
TokenError: 
async def drive(self,
a,
b,
c,



In [ ]:
# | export

AUTO_GEN_TXT = "Note: The above docstring is autogenerated by docstring-gen library!\n"


def _inject_docstring_to_source(
    source: str,
    docstring: str,
    lineno: int,
    node_col_offset: int,
    include_auto_gen_txt: bool,
) -> str:
    """Injects a docstring into a source code string.
     
    Args:
        source: The source code string.
        docstring: The docstring to inject.
        lineno: The line number to inject the docstring.
        node_col_offset: The column offset of the node.
        include_auto_gen_txt: Whether to include the auto-generated text.
     
    Returns:
        The source code string with the docstring injected.


    Note: The above docstring is autogenerated by docstring-gen library!
    """
    lineno = _get_lineno_to_append_docstring(source, lineno)
    lines = source.split("\n")
    indented_docstring = "\n".join(
        [
            line
            if i == 0 or i == len(docstring.split("\n")) - 1
            else f"{' ' * (node_col_offset + 4)}{line}"
            for i, line in enumerate(docstring.split("\n"))
        ]
    )
    indent = node_col_offset + 4
    nl = '\n'
    auto_gen_txt = f'{nl + nl + (" " * indent + AUTO_GEN_TXT) if include_auto_gen_txt else ""}'
    lines.insert(
        lineno,
        f'{" " * indent}"""{indented_docstring}{auto_gen_txt}{" " * indent}"""',
    )
    return "\n".join(lines)

In [ ]:
source = """
async def drive(
self,
a,
b,
c
):
    print(f'The {self.model} is now driving.')
"""

docstring = """Sample docstring

Args:
    s: sample args

Returns:
    sample return
"""

expected = """
async def drive(
self,
a,
b,
c
):
    \"""Sample docstring
    
    Args:
        s: sample args
    
    Returns:
        sample return


    Note: The above docstring is autogenerated by docstring-gen library!
    \"""
    print(f'The {self.model} is now driving.')
"""

lineno = 2
node_col_offset = 0
include_auto_gen_txt = True
actual = _inject_docstring_to_source(source, docstring, lineno, node_col_offset, include_auto_gen_txt)
print(actual)

assert actual == expected


async def drive(
self,
a,
b,
c
):
    """Sample docstring
    
    Args:
        s: sample args
    
    Returns:
        sample return


    Note: The above docstring is autogenerated by docstring-gen library!
    """
    print(f'The {self.model} is now driving.')



In [ ]:
source = """
async def drive(self):
    print(f'The {self.model} is now driving.')
"""

docstring = """Sample docstring

Args:
    s: sample args

Returns:
    sample return
"""

expected = """
async def drive(self):
    \"""Sample docstring
    
    Args:
        s: sample args
    
    Returns:
        sample return
    \"""
    print(f'The {self.model} is now driving.')
"""

lineno = 2
node_col_offset = 0
include_auto_gen_txt = False
actual = _inject_docstring_to_source(source, docstring, lineno, node_col_offset, include_auto_gen_txt)
print(actual)

assert actual == expected


async def drive(self):
    """Sample docstring
    
    Args:
        s: sample args
    
    Returns:
        sample return
    """
    print(f'The {self.model} is now driving.')



In [ ]:
# | export

PROMPT_TEMPLATE = '''
# Python 3.7

{source}

{prompt}
"""
'''

DEFAULT_PROMPT = "# An elaborate, high quality docstring in Google style for the above function:"

def _generate_docstring_using_codex(source: str, **kwargs) -> str:
    """Generate a docstring for a function using the OpenAI API.
     
    Args:
        source (str): The source code of the function.
        prompt (str): The prompt to use for the docstring.
        engine (str): The engine to use.
        temperature (float): The temperature to use.
        top_p (float): The top_p to use.
        max_tokens (int): The max_tokens to use.
        stop (str): The stop token to use.
        stream (bool): Whether to stream the response.
        logprobs (int): The number of logprobs to return.
        n (int): The number of completions to return.
        frequency_penalty (float): The frequency penalty to use.
        presence_penalty (float): The presence penalty to use.
        best_of (int): The number of best completions to return.
        logprobs (int): The number of logprobs to return.
        stop (str): The stop token to use.
        stream (bool): Whether to stream the response.
    frequency

    Note: The above docstring is autogenerated by docstring-gen library!
    """
    prompt = DEFAULT_PROMPT if kwargs["prompt"] is None else kwargs["prompt"]
    kwargs["prompt"] = PROMPT_TEMPLATE.format(source=source, prompt=prompt)
    try:
        response = completions_with_backoff(**kwargs)

    except openai.error.AuthenticationError as e:
        raise openai.error.AuthenticationError(
            "No API key provided. Please set the API key in the environment variable OPENAI_API_KEY=<API-KEY>. You can generate API keys in the OpenAI web interface. See https://onboard.openai.com for details."
        )

    ret_val: str = response.choices[0].text
    return ret_val

In [ ]:
source = """def add(x, y):
    return x + y
"""
tree = ast.parse(source)
node = tree.body[0]

docstring = _generate_docstring_using_codex(
    source,
    model="code-davinci-002",
    temperature=0,
    max_tokens=150,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    stop=["#", '"""'],
    prompt = None
)
source_with_docstring = _inject_docstring_to_source(
    source, docstring, node.lineno, node.col_offset, include_auto_gen_txt=True
)

print(source_with_docstring)
assert ast.get_docstring(ast.parse(source_with_docstring).body[0]) is not None

def add(x, y):
    """This function adds two numbers.
    
    Args:
        x: An int.
        y: An int.
    
    Returns:
        The sum of x and y.


    Note: The above docstring is autogenerated by docstring-gen library!
    """
    return x + y



In [ ]:
@contextmanager
def unset_env_var(name: str):
    """unset_env_var(name: str)
    
    Unset an environment variable and/or the OpenAI API key.
    
    This function is a context manager that unsets an environment variable and/or
    the OpenAI API key.
    
    Args:
        name: The name of the environment variable to unset.
    
    Returns:
        None
    
    Raises:
        None
    
    Examples:
        >>> with unset_env_var("OPENAI_API_KEY"):
        ...     openai.api_key
        ...
        None


    Note: The above docstring is autogenerated by docstring-gen library!
    """
    env_var_defined = name in os.environ
    api_key_set_in_openai = openai.api_key is not None
    if env_var_defined:
        original_value = os.environ.get(name)
        del os.environ[name]

    if api_key_set_in_openai:
        original_api_key = openai.api_key
        openai.api_key = None
    try:
        yield
    finally:
        if env_var_defined:
            os.environ[name] = original_value
        if api_key_set_in_openai:
            openai.api_key = original_api_key


source = """def add(x, y):
    return x + y
"""
tree = ast.parse(source)
node = tree.body[0]

with pytest.raises(openai.error.AuthenticationError) as e:
    with unset_env_var("OPENAI_API_KEY"):
        _generate_docstring_using_codex(
            source,
            model="code-davinci-002",
            temperature=0,
            max_tokens=150,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0,
            stop=["#", '"""'],
            prompt=None,
        )

print(e.value)
assert (
    "Please set the API key in the environment variable OPENAI_API_KEY=<API-KEY>"
    in str(e.value)
)

No API key provided. Please set the API key in the environment variable OPENAI_API_KEY=<API-KEY>. You can generate API keys in the OpenAI web interface. See https://onboard.openai.com for details.


In [ ]:
# | export

def _add_docstring(
    source: str,
    node: Union[ast.ClassDef, ast.FunctionDef, ast.AsyncFunctionDef],
    line_offset: int,
    include_auto_gen_txt: bool,
    **kwargs
) -> Tuple[str, int]:
    """    Adds a docstring to the given node in the source code.
        
        Args:
            source (str): The source code.
            node (Union[ast.ClassDef, ast.FunctionDef, ast.AsyncFunctionDef]): The node to which the docstring is to be added.
            line_offset (int): The line offset of the source code.
            include_auto_gen_txt (bool): Whether to include the text "Auto-generated by Codex" in the docstring.
            **kwargs: Keyword arguments to be passed to the function _generate_docstring_using_codex.
        
        Returns:
            Tuple[str, int]: A tuple containing the source code with the docstring added and the line offset.


    Note: The above docstring is autogenerated by docstring-gen library!
    """

    line_no = node.lineno + line_offset
    
    # Fix for ast's node.lineno giving line number of decorator 
    # instead of function/class definition in Python 3.7
    # Delete the below line once support for Python 3.7 is dropped
    line_no = _get_start_line_for_class_or_func(source, line_no)
    
    if hasattr(node, 'end_lineno') and node.end_lineno is not None:
        end_line_no = node.end_lineno + line_offset
    else:
        end_line_no = _calculate_end_lineno(source, line_no)
    
    code = _get_code_from_source(source, line_no, end_line_no)
    docstring = _generate_docstring_using_codex(code, **kwargs)
    
    source = _inject_docstring_to_source(source, docstring, line_no, node.col_offset, include_auto_gen_txt)
    line_offset += len(docstring.split("\n")) if not include_auto_gen_txt else len(docstring.split("\n")) + 3
    return source, line_offset

In [ ]:
source = """
class test:
    CONST_VAL = 1
    def __init__(self, a):
        self.a = a
        
    async def drive(self):
        print(f'The {self.model} is now driving.')
"""

tree = ast.parse(source)
line_offset = 0
for node in tree.body:
    source, line_offset = _add_docstring(
        source,
        node,
        line_offset,
        include_auto_gen_txt= True,
        model="code-davinci-002",
        temperature=0,
        max_tokens=150,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        stop=["#", '"""'],
        prompt=None,
    )

    for f in node.body:
        if not isinstance(f, (ast.FunctionDef, ast.AsyncFunctionDef)):
            continue
        source, line_offset = _add_docstring(
            source,
            f,
            line_offset,
            include_auto_gen_txt= True,
            model="code-davinci-002",
            temperature=0,
            max_tokens=150,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0,
            stop=["#", '"""'],
            prompt=None,
        )
        
def assert_docstring(source):
    """assert_docstring(source)
     
        Assert that the given source code has docstrings for all functions and classes.
     
        Args:
            source (str): The source code to check.


    Note: The above docstring is autogenerated by docstring-gen library!
    """
    tree = ast.parse(source)
    for node in tree.body:
        if isinstance(node, (ast.ClassDef, ast.FunctionDef, ast.AsyncFunctionDef)):
            assert ast.get_docstring(node) is not None
        if isinstance(node, ast.ClassDef):
            for f in node.body:
                if isinstance(f, (ast.FunctionDef, ast.AsyncFunctionDef)):
                    assert ast.get_docstring(node) is not None

    print(source)
    
assert_docstring(source)


class test:
    """    This is a test class.
        Args:
            a (int): The first parameter.
        Returns:
            bool: The return value. True for success, False otherwise.


    Note: The above docstring is autogenerated by docstring-gen library!
    """
    CONST_VAL = 1
    def __init__(self, a):
        """    This is a constructor for the class.
         
            Args:
                a (int): An integer.
         
            Returns:
                None


        Note: The above docstring is autogenerated by docstring-gen library!
        """
        self.a = a
        
    async def drive(self):
        """    This function is used to drive the car.
         
            Args:
                self: The car object.
         
            Returns:
                None


        Note: The above docstring is autogenerated by docstring-gen library!
        """
        print(f'The {self.model} is now driving.')



In [ ]:
# | export


def _check_and_add_docstrings_to_source(source: str, include_auto_gen_txt: bool, **kwargs) -> str:
    """    Checks and adds docstrings to the source code.
    
        Args:
            source (str): The source code to be checked and modified.
            include_auto_gen_txt (bool): Whether to include the auto-generated text in the docstring.
            **kwargs: Additional keyword arguments.
    
        Returns:
            str: The modified source code.


    Note: The above docstring is autogenerated by docstring-gen library!
    """

    tree = ast.parse(source)
    line_offset = 0

    for node in tree.body:
        if not isinstance(node, (ast.ClassDef, ast.FunctionDef, ast.AsyncFunctionDef)):
            continue
        if ast.get_docstring(node) is not None:
            continue

        # A class or a function without docstring
        source, line_offset = _add_docstring(source, node, line_offset, include_auto_gen_txt, **kwargs)
        if not isinstance(node, ast.ClassDef):
            continue
        # Is a class and we need to check the functions inside
        # 29 - 36 make it as a recursive function
        for f in node.body:
            if not isinstance(f, (ast.FunctionDef, ast.AsyncFunctionDef)):
                continue
            if ast.get_docstring(f) is not None:
                continue

            # should be a function inside the class for which there is no docstring
            source, line_offset = _add_docstring(source, f, line_offset, include_auto_gen_txt, **kwargs)

    return source

In [ ]:
source = """
def decorator1(func):
    def inner():
        func()
    return inner

def decorator2(func):
    def inner():
        func()
    return inner

@decorator1
@decorator2
def outer_func():
    def inner_func():
        print("Hello, World!")
    inner_func()
"""
actual = _check_and_add_docstrings_to_source(
    source,
    include_auto_gen_txt = True,
    model="code-davinci-002",
    temperature=0,
    max_tokens=150,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    stop=["#", '"""'],
    prompt=None,
)
assert_docstring(actual)

Note: Rate limit reached for default-codex. Command will automatically retry in 2 seconds.

def decorator1(func):
    """This is a decorator function.


    Note: The above docstring is autogenerated by docstring-gen library!
    """
    def inner():
        func()
    return inner

def decorator2(func):
    """This is a decorator function.


    Note: The above docstring is autogenerated by docstring-gen library!
    """
    def inner():
        func()
    return inner

@decorator1
@decorator2
def outer_func():
    """This is an example of a docstring in Google style.
     
    Args:
        None
     
    Returns:
        None


    Note: The above docstring is autogenerated by docstring-gen library!
    """
    def inner_func():
        print("Hello, World!")
    inner_func()



In [ ]:
# | export


def _get_files(nb_path: Path) -> List[Path]:
    """Get all Python files and notebooks in a directory.
    
    Parameters
    ----------
    nb_path : Path
        The path to the directory containing the notebooks.
    
    Returns
    -------
    List[Path]
        A list of paths to the Python files and notebooks.
    
    Raises
    ------
    ValueError
        If the directory does not contain any Python files or notebooks.


    Note: The above docstring is autogenerated by docstring-gen library!
    """
    exts = [".ipynb", ".py"]
    files = [
        f
        for f in nb_path.rglob("*")
        if f.suffix in exts
        and not any(p.startswith(".") for p in f.parts)
        and not f.name.startswith("_")
    ]
    
    if len(files) == 0:
        raise ValueError(f"The directory {nb_path.resolve()} does not contain any Python files or notebooks")

    return files

In [ ]:
with TemporaryDirectory() as d:
    nbs_path = Path(d) / "nbs"
    nbs_path.mkdir(parents=True)
    
    hidden_dir = nbs_path / ".hidden"
    hidden_dir.mkdir(parents=True)

    shutil.copyfile(Path("..") / "settings.ini", nbs_path / "settings.ini")
    shutil.copyfile(Path("..") / "fixtures" / "Test_Data.ipynb", nbs_path / "_test.ipynb")
    shutil.copyfile(Path("..") / "fixtures" / "Test_Data.ipynb", nbs_path / "test.ipynb")
    shutil.copyfile(Path("..") / "fixtures" / "Test_Data.ipynb", nbs_path / "test_1.ipynb")
    
    shutil.copyfile(Path("..") / "fixtures" / "Test_Data.ipynb", hidden_dir / "test.ipynb")
    shutil.copyfile(Path("..") / "fixtures" / "Test_Data.ipynb", hidden_dir / "test_1.ipynb")
    
    for f in nbs_path.rglob("*"):
        print(f)

    files = _get_files(nbs_path)

    assert len(files) == 2
    print(f"\n\n{files}")
    assert files == [nbs_path / "test_1.ipynb", nbs_path / "test.ipynb"]
    
    

/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpofiek56g/nbs/.hidden
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpofiek56g/nbs/test_1.ipynb
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpofiek56g/nbs/settings.ini
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpofiek56g/nbs/test.ipynb
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpofiek56g/nbs/_test.ipynb
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpofiek56g/nbs/.hidden/test_1.ipynb
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpofiek56g/nbs/.hidden/test.ipynb


[PosixPath('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpofiek56g/nbs/test_1.ipynb'), PosixPath('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpofiek56g/nbs/test.ipynb')]


In [ ]:
with pytest.raises(ValueError) as e:

    with TemporaryDirectory() as d:
        nbs_path = Path(d) / "nbs"
        nbs_path.mkdir(parents=True)

        _get_files(nbs_path)
        
print(e.value)

The directory /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp8pd1yd8i/nbs does not contain any Python files or notebooks


In [ ]:
# | export


def _add_docstring_to_nb(
    file: Path, version: int, include_auto_gen_txt: bool, **kwargs
):
    """Adds docstrings to all functions in a Jupyter notebook.
     
    Parameters
    ----------
    file : Path
        The path to the Jupyter notebook.
    version : int
        The version of the Jupyter notebook.
    include_auto_gen_txt : bool
        Whether to include the text "Auto-generated by ..." in the docstring.
    kwargs : dict
        Keyword arguments to be passed to the function
        `_check_and_add_docstrings_to_source`.
     
    Returns
    -------
    None
     
    Raises
    ------
    None
     
    Notes
    -----
    None
     
    Examples
    --------
    None


    Note: The above docstring is autogenerated by docstring-gen library!
    """
    _f = nbformat.read(file, as_version=version)
    for cell in _f.cells:
        if cell.cell_type == "code":
            cell["source"] = _check_and_add_docstrings_to_source(
                cell["source"], include_auto_gen_txt, **kwargs
            )
    nbformat.write(_f, file)


def _add_docstring_to_py(file: Path, include_auto_gen_txt: bool, **kwargs):
    """Adds docstrings to all functions and classes in a Python file.
     
    :param file: The file to add docstrings to.
    :param include_auto_gen_txt: Whether to include the "This file was auto-generated" text.
    :param kwargs: Additional keyword arguments to pass to the docstring generator.


    Note: The above docstring is autogenerated by docstring-gen library!
    """
    with file.open("r") as f:
        source = f.read()
    source = _check_and_add_docstrings_to_source(source, include_auto_gen_txt, **kwargs)
    with file.open("w") as f:
        f.write(source)


def add_docstring_to_source(
    path: Union[str, Path],
    version: int = 4,
    include_auto_gen_txt: bool = True,
    model: str = "code-davinci-002",
    temperature: int = 0,
    max_tokens: int = 250,
    top_p: float = 1.0,
    frequency_penalty: float = 0.0,
    presence_penalty: float = 0.0,
    stop: List[str] = ["#", '"""'],
    prompt: Optional[str] = None,
):
    """Adds a docstring to the source code.
    
    This function adds a docstring to the source code.
    
    Args:
        path: The path to the source code.
        version: The version of the docstring.
        include_auto_gen_txt: Whether to include the text "Auto-generated by Code-DaVinci".
        model: The model to use.
        temperature: The temperature to use.
        max_tokens: The maximum number of tokens to use.
        top_p: The top p to use.
        frequency_penalty: The frequency penalty to use.
        presence_penalty: The presence penalty to use.
        stop: The stop tokens to use.
        prompt: The prompt to use.
    
    Returns:
        None
    
    Raises:
        ValueError: If the path is not a directory or a Python file.


    Note: The above docstring is autogenerated by docstring-gen library!
    """

    path = Path(path)
    files = _get_files(path) if path.is_dir() else [path]

    for file in files:
        if file.suffix == ".ipynb":
            _add_docstring_to_nb(
                file=file,
                version=version,
                include_auto_gen_txt=include_auto_gen_txt,
                model=model,
                temperature=temperature,
                max_tokens=max_tokens,
                top_p=top_p,
                frequency_penalty=frequency_penalty,
                presence_penalty=presence_penalty,
                stop=stop,
                prompt=prompt,
            )
        else:
            _add_docstring_to_py(
                file=file,
                include_auto_gen_txt=include_auto_gen_txt,
                model=model,
                temperature=temperature,
                max_tokens=max_tokens,
                top_p=top_p,
                frequency_penalty=frequency_penalty,
                presence_penalty=presence_penalty,
                stop=stop,
                prompt=prompt,
            )

In [ ]:
with TemporaryDirectory() as d:
    nbs_path = Path(d) / "nbs"
    nbs_path.mkdir(parents=True)

    shutil.copyfile(Path("..") / "fixtures" / "Test_Data.ipynb", nbs_path / "test.ipynb")
    shutil.copyfile(Path("..") / "fixtures" / "Test_Data.ipynb", nbs_path / "_test.ipynb")
    
    shutil.copyfile(Path("..") / "fixtures" / "test_data.py", nbs_path / "test_data.py")
    shutil.copyfile(Path("..") / "settings.ini", nbs_path / "settings.ini")

    add_docstring_to_source(nbs_path)
    
    with (nbs_path / "test.ipynb").open("r") as f:
        nb = nbformat.read(f, as_version=4)

for cell in nb.cells:
#     print(cell["source"])
    
    if cell.cell_type == "code":
        tree = ast.parse(cell["source"])        
        print(cell["source"])
        for node in tree.body:
            if not isinstance(node, (ast.ClassDef, ast.FunctionDef, ast.AsyncFunctionDef)):
                continue
            assert ast.get_docstring(node)
            
    else:
        print(cell["source"])

# Test notebook for docstring generator

> Test notebook for docstring generator
# | export

from typing import *
import os
from pathlib import Path
# from contextlib import contextmanager

from contextlib import contextmanager

import shutil
from tempfile import TemporaryDirectory

# Title
# | export

# Vehicle class
class Vehicle:
    """    This function will drive the vehicle.
        
        Args:
            None
        
        Returns:
            None
        
        Raises:
            None


    Note: The above docstring is autogenerated by docstring-gen library!
    """
    # Constructor function
    def __init__(self, brand, model, type):
        """Constructor function
        
        Args:
            brand: Vehicle's brand
            model: Vehicle's model
            type: Vehicle's type
        """
        self.brand = brand
        self.model = model
        self.type = type
        self.gas_tank_size = 14
        self.fuel_level = 0
    
    # fuel_up function
